## Data Cleaning and Preparation

###### Handling Missing Data

The way that missing data is represented in pandas objects is somewhat imperfect,
but it is functional for a lot of users. For numeric data, pandas uses the floating-point
value NaN (Not a Number) to represent missing data. We call this a sentinel value that
can be easily detected

In [1]:
import pandas as pd
import numpy as np

In [3]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])
string_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [7]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

In [13]:
# The built-in Python None value is also treated as NA in object arrays
string_data[0] = None
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

In [14]:
string_data.notnull()

0    False
1     True
2    False
3     True
dtype: bool

##### Filtering Out Missing Data

There are a few ways to filter out missing data. While you always have the option to
do it by hand using pandas.isnull and boolean indexing, the dropna can be helpful.
On a Series, it returns the Series with only the non-null data and index values

In [19]:
from numpy import nan as NA

In [20]:
data = pd.Series([1, NA, 3.5, NA, 7])

In [23]:
cleaned = data.dropna()
cleaned

0    1.0
2    3.5
4    7.0
dtype: float64

In [27]:
# This is equivalent to previous example. Here Boolean indexing is used.
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

With DataFrame objects, things are a bit more complex. You may want to drop rows
or columns that are all NA or only those containing any NAs. dropna by default drops
any row containing a missing value


In [33]:
data = pd.DataFrame([[1.,6.5, 3.], [1., NA, NA], [NA, NA, NA], [NA, 6.5, 3.]])
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [40]:
cleaned = data.dropna()
cleaned

,0,1,2
0,1.0,6.5,3.0


In [36]:
# Passing how='all' will only drop rows that are all NA
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [43]:
# To drop columns in the same way, pass axis=1:
data[4] = NA

In [44]:
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [45]:
data.dropna(how='all', axis=1)

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


A related way to filter out DataFrame rows tends to concern time series data. Suppose
you want to keep only rows containing a certain number of observations. We can
indicate this with the thresh argument

In [47]:
df = pd.DataFrame(np.random.randn(7,3))
df

,0,1,2
0,1.202119,0.437342,-0.083514
1,0.192650,-0.417913,1.931056
2,-0.360667,-1.721715,1.997239
3,-0.173243,-0.859095,-0.714570
4,1.048893,-0.423915,-0.364243
5,-0.095055,-0.602152,-1.053918
6,-0.349954,-0.252045,-0.422060


In [52]:
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA
df

,0,1,2
0,1.202119,NaN,NaN
1,0.192650,NaN,NaN
2,-0.360667,NaN,1.997239
3,-0.173243,NaN,-0.714570
4,1.048893,-0.423915,-0.364243
5,-0.095055,-0.602152,-1.053918
6,-0.349954,-0.252045,-0.422060


In [53]:
df.dropna()

,0,1,2
4,1.048893,-0.423915,-0.364243
5,-0.095055,-0.602152,-1.053918
6,-0.349954,-0.252045,-0.422060


In [61]:
df.dropna(thresh=2)

,0,1,2
2,-0.360667,NaN,1.997239
3,-0.173243,NaN,-0.714570
4,1.048893,-0.423915,-0.364243
5,-0.095055,-0.602152,-1.053918
6,-0.349954,-0.252045,-0.422060


In [59]:
df.dropna(axis=1)

,0
0,1.202119
1,0.192650
2,-0.360667
3,-0.173243
4,1.048893
5,-0.095055
6,-0.349954


##### Filling In Missing Data

In [62]:
# For most pur‐poses, the fillna method is the workhorse function to use
df

,0,1,2
0,1.202119,NaN,NaN
1,0.192650,NaN,NaN
2,-0.360667,NaN,1.997239
3,-0.173243,NaN,-0.714570
4,1.048893,-0.423915,-0.364243
5,-0.095055,-0.602152,-1.053918
6,-0.349954,-0.252045,-0.422060


In [65]:
df.fillna(0)

,0,1,2
0,1.202119,0.000000,0.000000
1,0.192650,0.000000,0.000000
2,-0.360667,0.000000,1.997239
3,-0.173243,0.000000,-0.714570
4,1.048893,-0.423915,-0.364243
5,-0.095055,-0.602152,-1.053918
6,-0.349954,-0.252045,-0.422060


In [66]:
# Calling fillna with a dict, we can use a different fill value for each column
df.fillna({1:0.5, 2:0})

,0,1,2
0,1.202119,0.500000,0.000000
1,0.192650,0.500000,0.000000
2,-0.360667,0.500000,1.997239
3,-0.173243,0.500000,-0.714570
4,1.048893,-0.423915,-0.364243
5,-0.095055,-0.602152,-1.053918
6,-0.349954,-0.252045,-0.422060


In [69]:
# fillna returns a new object, but we can modify the existing object in-place
df.fillna(0, inplace=True)
df

,0,1,2
0,1.202119,0.000000,0.000000
1,0.192650,0.000000,0.000000
2,-0.360667,0.000000,1.997239
3,-0.173243,0.000000,-0.714570
4,1.048893,-0.423915,-0.364243
5,-0.095055,-0.602152,-1.053918
6,-0.349954,-0.252045,-0.422060


In [82]:
# The same interpolation methods available for reindexing can be used with fillna
df = pd.DataFrame(np.random.rand(6,3))
df

,0,1,2
0,0.926834,0.297345,0.454495
1,0.548999,0.332209,0.242970
2,0.674743,0.585748,0.300526
3,0.672332,0.804326,0.063339
4,0.938367,0.639185,0.469232
5,0.106785,0.156610,0.313239


In [83]:
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA
df

,0,1,2
0,0.926834,0.297345,0.454495
1,0.548999,0.332209,0.242970
2,0.674743,NaN,0.300526
3,0.672332,NaN,0.063339
4,0.938367,NaN,NaN
5,0.106785,NaN,NaN


In [84]:
df.fillna(method='ffill')

,0,1,2
0,0.926834,0.297345,0.454495
1,0.548999,0.332209,0.242970
2,0.674743,0.332209,0.300526
3,0.672332,0.332209,0.063339
4,0.938367,0.332209,0.063339
5,0.106785,0.332209,0.063339


In [88]:
df.fillna(method='ffill', limit=2)

,0,1,2
0,0.926834,0.297345,0.454495
1,0.548999,0.332209,0.242970
2,0.674743,0.332209,0.300526
3,0.672332,0.332209,0.063339
4,0.938367,NaN,0.063339
5,0.106785,NaN,0.063339


In [91]:
# With fillna we can do lots of other things with a little creativity. We might pass the mean or median value of a Series
data = pd.Series([1., NA, 3.5, NA, 7])
data

0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64

In [92]:
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64